<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#demograhpy" data-toc-modified-id="demograhpy-1">demograhpy</a></span></li><li><span><a href="#paybacks" data-toc-modified-id="paybacks-2">paybacks</a></span><ul class="toc-item"><li><span><a href="#ECA" data-toc-modified-id="ECA-2.1">ECA</a></span></li><li><span><a href="#SERAFE" data-toc-modified-id="SERAFE-2.2">SERAFE</a></span></li></ul></li><li><span><a href="#tax-incomes" data-toc-modified-id="tax-incomes-3">tax incomes</a></span><ul class="toc-item"><li><span><a href="#cars_fuels" data-toc-modified-id="cars_fuels-3.1">cars_fuels</a></span></li><li><span><a href="#circulation" data-toc-modified-id="circulation-3.2">circulation</a></span></li><li><span><a href="#computation-of-tax" data-toc-modified-id="computation-of-tax-3.3">computation of tax</a></span></li></ul></li></ul></div>

In [38]:
# modules for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

# module to save space
import gc

import numpy as np

As we start from *Switzerland*, and in particular canton **Vaud** the case stydy all costs are in  **CHF/year**

Also, instead of start thinking from how to _get_ money, we think how to give back the tax first

# demograhpy   

basic demographic data, relevant for our case study (Switzerland)

(these N are a taken just as a starting point -- relative to past statistics) -- sources:


[population total](https://www.atlas.bfs.admin.ch/maps/13/fr/14459_72_71_70/23029.html)   
[households N](http://www.scris.vd.ch/Data_Dir/ElementsDir/8235/2/F/Portraits-des-familles.pdf)

In [4]:
pop_vaud_2019 = 799145

households_vaud_2013 = 323064

# paybacks

we need to return that money immediately to citizen, to 
- build trust in the carbon reform 
- create a sustainable reform

the first principle here is avoid at the beginning too much differences between residents, in particular countryside/city, where there is low/high public transport frequency 

There, the idea is to start payback on tax/public services that everybody needs (both in villages and cities)

## ECA

Given the increasing evidence of the relation between *pollution > increase frequency for sever weather events*, it makes that costs from natural disasters are payed by who pollutes

in canton **Vaud**: one idea would be to stop paying the basic fee for the [ECA](https://www.eca-vaud.ch)

which is the public intervention service/insurance against natural disasters 

In [26]:
eca_basic_fee = 20

eca_basic_tot = eca_basic_fee * households_vaud_2013
eca_basic_tot

6461280

## SERAFE

it might seems counter-intuitive, but paying the public news/radio-TV services might be also interesting, as this is a tax _every household_ has to pay, no matter where you live in the country

In [27]:
serafe_fee = 365

serafe_tot = serafe_fee * households_vaud_2013
serafe_tot

117918360

# tax incomes

the general idea is to tax fossil fuels usage

statistics on mobility: 
(mind that in EN sometimes you don't find everything - better in FR/DE)
    
https://www.bfs.admin.ch/bfs/fr/home/statistiques/mobilite-transports.html 


[average daily road traffic](https://www.bfs.admin.ch/bfs/fr/home/statistiques/mobilite-transports/infrastructures-transport-vehicules/comptage-suisse-circulation-route.html)

[costs-funding](https://www.bfs.admin.ch/bfs/en/home/statistics/mobility-transport/costs-funding.html)

[N vehicles](https://www.bfs.admin.ch/bfs/fr/home/statistiques/mobilite-transports/infrastructures-transport-vehicules/vehicules/vehicules-routiers-parc-taux-motorisation.html)

## cars_fuels

[petrol data](https://www.globalpetrolprices.com/Switzerland/gasoline_prices/)

In [ ]:
gasoline_per_litre = 1.4 

n_cars_vaud = 416941 # this seems a reasonable lower bound, based on https://www.bfs.admin.ch/bfs/fr/home/statistiques/mobilite-transports/infrastructures-transport-vehicules/comptage-suisse-circulation-route.html

from this stats > [av. cost per vehicle](https://www.bfs.admin.ch/bfs/fr/home/statistiques/mobilite-transports/couts-financement/route-mobilite-douce.html)

In [19]:
## all are YEARLY averaged estimates -- by the OFSP

cost_vehicle = 7620 # av. annual cost / vehi. 
cost_vehicle_km = 0.6 # av. annual cost / (vehi.*km) 
km_per_vehicle = cost_vehicle / cost_vehicle_km
km_per_vehicle

12700.0

In [12]:
litres_car_year = 5 * 356 #??? # just to get some Numbers > find stats!

In [30]:
def fuel_tax_tot(tax_per_litre):
    
    '''
    enter tax in CHF -- for now 
    '''
    return tax_per_litre * litres_car_year * n_cars_vaud

## circulation

In [31]:
def circ_tax_tot(circ_tax):
    
    return n_cars_vaud * circ_tax

## computation of tax

clearly, we need (N-1) _more_ constraints -- with N = number of tax income sources

one way, would be to set the ratio among total incomes -- how much do we want each source to contribute?

for the time being, we can keep it to a simple linear problem:

`Ax=b`, where 
`x`: unitary taxes (_increments_) 

In [74]:
circ_to_fuel_ratio = 1  # just a N for now

# circ_tax_tot(circ_tax) - circ_to_fuel_ratio * fuel_tax_tot(fuel_tax)  = 0 ## in case of non-linearity

b = np.array([0, eca_basic_tot + serafe_tot])

coeff_matrix = np.array(
    [[n_cars_vaud, -circ_to_fuel_ratio * litres_car_year * n_cars_vaud],
     [n_cars_vaud, litres_car_year * n_cars_vaud]])
print(coeff_matrix)

circ_tax, fuel_tax = np.linalg.solve(coeff_matrix, b)


print('\n\nassuming a circ_to_fuel_ratio:', circ_to_fuel_ratio)
print('(extra) circulation tax:', np.round(circ_tax, 3), 'chf/(y*veh)')
print('(extra) fuel tax:', np.round(fuel_tax, 3), 'chf/l')

[[    416941 -742154980]
 [    416941  742154980]]


assuming a circ_to_fuel_ratio: 1
(extra) circulation tax: 149.157 chf/(y*veh)
(extra) fuel tax: 0.084 chf/l


(in case the problem becomes non linear > https://docs.scipy.org/doc/scipy/reference/optimize.nonlin.html)